# Router RAG Agent - 커피 키오스크 주문 시스템

## 1. 환경 설정 및 라이브러리 임포트

In [6]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

True

In [3]:
# 필요한 라이브러리 설치
%pip install -q openai pydantic

In [7]:
import os
import json
from typing import List, Dict, Optional, Literal
from datetime import datetime
from openai import OpenAI
from pydantic import BaseModel, Field, validator
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# OpenAI 클라이언트 초기화
client = OpenAI()

## 2. Pydantic 모델 정의

In [8]:
class RouteClassification(BaseModel):
    """라우터가 질문을 분류한 결과를 표현하는 모델"""
    route_type: Literal["menu", "recipe", "price"] = Field(
        description="질문의 유형: menu(메뉴 정보), recipe(레시피 정보), price(가격/프로모션)"
    )
    confidence: float = Field(
        ge=0.0, le=1.0,
        description="분류에 대한 신뢰도 (0.0 ~ 1.0)"
    )
    reasoning: str = Field(
        description="해당 라우트로 분류한 이유"
    )


class DocumentChunk(BaseModel):
    """RAG 데이터베이스의 문서 청크를 표현하는 모델"""
    id: str = Field(description="문서 청크의 고유 ID")
    content: str = Field(description="문서 청크의 내용")
    category: str = Field(description="문서가 속한 카테고리")
    metadata: Dict = Field(default_factory=dict, description="추가 메타데이터")
    embedding: Optional[List[float]] = Field(default=None, description="문서의 임베딩 벡터")


class RetrievalResult(BaseModel):
    """검색 결과를 표현하는 모델"""
    chunks: List[DocumentChunk] = Field(description="검색된 문서 청크 목록")
    scores: List[float] = Field(description="각 청크의 유사도 점수")
    query: str = Field(description="원본 검색 쿼리")
    
    @validator('scores')
    def validate_scores_length(cls, v, values):
        """검색 점수와 청크의 개수가 일치하는지 검증한다"""
        if 'chunks' in values and len(v) != len(values['chunks']):
            raise ValueError("검색 점수와 청크의 개수가 일치해야 한다")
        return v


class ConversationMessage(BaseModel):
    """대화 메시지를 표현하는 모델"""
    role: Literal["user", "assistant", "system"] = Field(description="메시지 역할")
    content: str = Field(description="메시지 내용")
    timestamp: datetime = Field(default_factory=datetime.now, description="메시지 생성 시간")
    metadata: Dict = Field(default_factory=dict, description="추가 메타데이터")


class AgentResponse(BaseModel):
    """에이전트의 최종 응답을 표현하는 모델"""
    answer: str = Field(description="사용자에게 제공되는 답변")
    route_used: str = Field(description="사용된 라우트")
    sources: List[str] = Field(description="답변 생성에 사용된 소스 문서 ID")
    confidence: float = Field(ge=0.0, le=1.0, description="답변에 대한 신뢰도")
    validation_passed: bool = Field(description="검증 통과 여부")
    feedback_score: Optional[float] = Field(default=None, description="피드백 점수")

/var/folders/v9/46y9d8bn1lxgjt7g439hsf8c0000gn/T/ipykernel_92317/1354236395.py:30: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('scores')


## 3. RAG 지식 베이스 구축

In [9]:
# 메뉴 정보 지식 베이스
MENU_KNOWLEDGE = [
    DocumentChunk(
        id="menu_001",
        content="아메리카노는 에스프레소에 물을 추가한 커피로, 진하고 깊은 맛이 특징이다. HOT과 ICE 옵션이 있다.",
        category="menu",
        metadata={"beverage_type": "coffee", "has_caffeine": True}
    ),
    DocumentChunk(
        id="menu_002",
        content="카페라떼는 에스프레소에 스팀 밀크를 넣은 커피로, 부드럽고 크리미한 맛이 특징이다. 우유의 비율이 높아 커피가 부담스러운 사람에게 적합하다.",
        category="menu",
        metadata={"beverage_type": "coffee", "has_caffeine": True, "contains_milk": True}
    ),
    DocumentChunk(
        id="menu_003",
        content="카푸치노는 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 커피다. 풍부한 거품과 고소한 맛이 특징이다.",
        category="menu",
        metadata={"beverage_type": "coffee", "has_caffeine": True, "contains_milk": True}
    ),
    DocumentChunk(
        id="menu_004",
        content="녹차라떼는 고품질 녹차 파우더와 우유를 블렌딩한 음료로, 녹차의 은은한 향과 우유의 부드러움이 조화롭다. 카페인이 적어 부담없이 즐길 수 있다.",
        category="menu",
        metadata={"beverage_type": "tea", "has_caffeine": True, "contains_milk": True}
    )
]

# 레시피 정보 지식 베이스
RECIPE_KNOWLEDGE = [
    DocumentChunk(
        id="recipe_001",
        content="아메리카노 레시피: 에스프레소 샷 2개를 추출한 후 물 150ml를 추가한다. ICE 아메리카노의 경우 얼음을 먼저 넣고 에스프레소와 물을 추가한다.",
        category="recipe",
        metadata={"difficulty": "easy", "prep_time": "2분"}
    ),
    DocumentChunk(
        id="recipe_002",
        content="카페라떼 레시피: 에스프레소 샷 2개를 추출하고, 우유 200ml를 스티밍하여 에스프레소에 부드럽게 붓는다. 우유 온도는 65-70도가 적절하다.",
        category="recipe",
        metadata={"difficulty": "medium", "prep_time": "3분"}
    ),
    DocumentChunk(
        id="recipe_003",
        content="카푸치노 레시피: 에스프레소 샷 1개, 스팀 밀크 60ml, 우유 거품 60ml를 순서대로 층을 이루어 넣는다. 거품은 미세하고 벨벳 같은 질감이 중요하다.",
        category="recipe",
        metadata={"difficulty": "hard", "prep_time": "4분"}
    )
]

# 가격 및 프로모션 정보 지식 베이스
PRICE_KNOWLEDGE = [
    DocumentChunk(
        id="price_001",
        content="아메리카노 가격: HOT 4,500원, ICE 5,000원. 사이즈 업그레이드 시 각각 500원 추가된다.",
        category="price",
        metadata={"currency": "KRW"}
    ),
    DocumentChunk(
        id="price_002",
        content="카페라떼와 카푸치노 가격: HOT 5,000원, ICE 5,500원. 두 음료 모두 동일한 가격이다.",
        category="price",
        metadata={"currency": "KRW"}
    ),
    DocumentChunk(
        id="price_003",
        content="이번 주 프로모션: 오후 2시~5시 사이 아메리카노 구매 시 20% 할인. 중복 할인은 불가능하다.",
        category="price",
        metadata={"promotion_type": "time_based", "valid_until": "2025-10-31"}
    ),
    DocumentChunk(
        id="price_004",
        content="녹차라떼 가격: HOT 5,500원, ICE 6,000원. 프리미엄 녹차를 사용하여 다른 라떼보다 가격이 높다.",
        category="price",
        metadata={"currency": "KRW"}
    )
]

# 전체 지식 베이스 통합
ALL_KNOWLEDGE = MENU_KNOWLEDGE + RECIPE_KNOWLEDGE + PRICE_KNOWLEDGE

print(f"총 {len(ALL_KNOWLEDGE)}개의 문서 청크가 로드되었다.")
print(f"메뉴 정보: {len(MENU_KNOWLEDGE)}개")
print(f"레시피 정보: {len(RECIPE_KNOWLEDGE)}개")
print(f"가격 정보: {len(PRICE_KNOWLEDGE)}개")

총 11개의 문서 청크가 로드되었다.
메뉴 정보: 4개
레시피 정보: 3개
가격 정보: 4개


## 4. Memory 시스템 구현

In [10]:
class ConversationMemory:
    """대화 이력을 관리하는 메모리 클래스"""
    
    def __init__(self, max_history: int = 10):
        """
        Args:
            max_history: 저장할 최대 대화 수 (메모리 관리를 위해 제한)
        """
        self.messages: List[ConversationMessage] = []
        self.max_history = max_history
    
    def add_message(self, role: str, content: str, metadata: Dict = None):
        """새로운 메시지를 메모리에 추가한다"""
        message = ConversationMessage(
            role=role,
            content=content,
            metadata=metadata or {}
        )
        self.messages.append(message)
        
        # 최대 이력 수를 초과하면 오래된 메시지부터 제거한다
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
    
    def get_context(self, include_system: bool = True) -> List[Dict]:
        """OpenAI API 형식으로 대화 이력을 반환한다"""
        context = []
        for msg in self.messages:
            if msg.role == "system" and not include_system:
                continue
            context.append({
                "role": msg.role,
                "content": msg.content
            })
        return context
    
    def clear(self):
        """모든 대화 이력을 삭제한다"""
        self.messages = []
    
    def get_summary(self) -> str:
        """현재까지의 대화를 요약한다"""
        if not self.messages:
            return "대화 이력이 없다."
        
        user_messages = [m for m in self.messages if m.role == "user"]
        assistant_messages = [m for m in self.messages if m.role == "assistant"]
        
        return f"총 {len(self.messages)}개의 메시지 (사용자: {len(user_messages)}, 에이전트: {len(assistant_messages)})"

# 메모리 인스턴스 생성
memory = ConversationMemory(max_history=10)
print("대화 메모리가 초기화되었다.")

대화 메모리가 초기화되었다.


## 5. RAG 검색 엔진 구현

In [11]:
class RAGEngine:
    """RAG 검색 엔진 클래스"""
    
    def __init__(self, documents: List[DocumentChunk]):
        """
        Args:
            documents: 검색할 문서 청크 리스트
        """
        self.documents = documents
        self.embeddings_cache = {}
        # 모든 문서에 대한 임베딩을 미리 생성한다
        self._initialize_embeddings()
    
    def _get_embedding(self, text: str) -> List[float]:
        """텍스트의 임베딩 벡터를 생성한다"""
        # 캐시 확인
        if text in self.embeddings_cache:
            return self.embeddings_cache[text]
        
        # OpenAI 임베딩 API 호출
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        embedding = response.data[0].embedding
        
        # 캐시에 저장
        self.embeddings_cache[text] = embedding
        return embedding
    
    def _initialize_embeddings(self):
        """모든 문서에 대한 임베딩을 초기화한다"""
        print("문서 임베딩을 생성하는 중...")
        for doc in self.documents:
            doc.embedding = self._get_embedding(doc.content)
        print(f"{len(self.documents)}개 문서의 임베딩이 생성되었다.")
    
    def search(self, query: str, category: Optional[str] = None, top_k: int = 3) -> RetrievalResult:
        """
        쿼리와 가장 유사한 문서를 검색한다
        
        Args:
            query: 검색 쿼리
            category: 특정 카테고리로 필터링 (None이면 전체 검색)
            top_k: 반환할 상위 문서 개수
        
        Returns:
            검색 결과를 담은 RetrievalResult 객체
        """
        # 쿼리 임베딩 생성
        query_embedding = self._get_embedding(query)
        query_vector = np.array(query_embedding).reshape(1, -1)
        
        # 카테고리 필터링
        candidate_docs = self.documents
        if category:
            candidate_docs = [doc for doc in self.documents if doc.category == category]
        
        # 유사도 계산
        doc_vectors = np.array([doc.embedding for doc in candidate_docs])
        similarities = cosine_similarity(query_vector, doc_vectors)[0]
        
        # 상위 k개 문서 선택
        top_indices = np.argsort(similarities)[::-1][:top_k]
        top_docs = [candidate_docs[i] for i in top_indices]
        top_scores = [float(similarities[i]) for i in top_indices]
        
        return RetrievalResult(
            chunks=top_docs,
            scores=top_scores,
            query=query
        )

# RAG 엔진 인스턴스 생성
rag_engine = RAGEngine(ALL_KNOWLEDGE)

문서 임베딩을 생성하는 중...
11개 문서의 임베딩이 생성되었다.


## 6. Function Calling & Tool 정의

In [12]:
# Function Calling을 위한 도구 정의
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_menu_info",
            "description": "메뉴 정보를 검색한다. 커피나 음료의 종류, 특징, 구성 요소에 대한 질문에 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "검색할 메뉴 관련 질문"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_recipe_info",
            "description": "레시피 정보를 검색한다. 음료 제조 방법, 재료, 조리 과정에 대한 질문에 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "검색할 레시피 관련 질문"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_price_info",
            "description": "가격 및 프로모션 정보를 검색한다. 가격, 할인, 이벤트에 대한 질문에 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "검색할 가격/프로모션 관련 질문"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

# 도구 실행 함수 매핑
TOOL_FUNCTIONS = {
    "search_menu_info": lambda query: rag_engine.search(query, category="menu", top_k=2),
    "search_recipe_info": lambda query: rag_engine.search(query, category="recipe", top_k=2),
    "search_price_info": lambda query: rag_engine.search(query, category="price", top_k=2)
}

print(f"총 {len(TOOLS)}개의 도구가 정의되었다.")

총 3개의 도구가 정의되었다.


## 7. Validation 시스템 구현

In [13]:
class ValidationSystem:
    """답변 검증 시스템 클래스"""
    
    def __init__(self, min_relevance_score: float = 0.5, min_confidence: float = 0.6):
        """
        Args:
            min_relevance_score: 검색 결과의 최소 유사도 점수
            min_confidence: 답변의 최소 신뢰도
        """
        self.min_relevance_score = min_relevance_score
        self.min_confidence = min_confidence
    
    def validate_retrieval(self, result: RetrievalResult) -> tuple[bool, str]:
        """
        검색 결과의 유효성을 검증한다
        
        Returns:
            (검증 통과 여부, 검증 메시지)
        """
        # 검색 결과가 없는 경우
        if not result.chunks:
            return False, "검색 결과가 없다"
        
        # 최고 유사도 점수 확인
        max_score = max(result.scores)
        if max_score < self.min_relevance_score:
            return False, f"검색 결과의 관련성이 낮다 (최고 점수: {max_score:.2f})"
        
        return True, "검색 결과가 유효하다"
    
    def validate_answer(self, answer: str, retrieval_result: RetrievalResult) -> tuple[bool, str]:
        """
        생성된 답변의 유효성을 검증한다
        
        Returns:
            (검증 통과 여부, 검증 메시지)
        """
        # 답변이 비어있는 경우
        if not answer or len(answer.strip()) < 10:
            return False, "답변이 너무 짧거나 비어있다"
        
        # 답변이 검색된 내용을 포함하는지 확인
        # 실제로는 더 정교한 검증이 필요하지만, 여기서는 간단히 키워드 포함 여부만 확인한다
        answer_lower = answer.lower()
        has_relevant_content = False
        
        for chunk in retrieval_result.chunks:
            # 검색된 문서의 주요 키워드가 답변에 포함되어 있는지 확인
            words = chunk.content.lower().split()
            relevant_words = [w for w in words if len(w) > 2][:5]
            if any(word in answer_lower for word in relevant_words):
                has_relevant_content = True
                break
        
        if not has_relevant_content:
            return False, "답변이 검색 결과와 관련이 없다"
        
        return True, "답변이 유효하다"
    
    def calculate_confidence(self, retrieval_result: RetrievalResult) -> float:
        """
        검색 결과를 바탕으로 신뢰도를 계산한다
        
        Returns:
            0.0 ~ 1.0 사이의 신뢰도 점수
        """
        if not retrieval_result.scores:
            return 0.0
        
        # 상위 2개 점수의 평균을 신뢰도로 사용
        top_scores = sorted(retrieval_result.scores, reverse=True)[:2]
        confidence = sum(top_scores) / len(top_scores)
        
        return min(confidence, 1.0)

# 검증 시스템 인스턴스 생성
validator = ValidationSystem(min_relevance_score=0.5, min_confidence=0.6)
print("검증 시스템이 초기화되었다.")

검증 시스템이 초기화되었다.


## 8. Recovery 메커니즘 구현

In [14]:
class RecoverySystem:
    """복구 메커니즘 시스템 클래스"""
    
    def __init__(self, rag_engine: RAGEngine):
        self.rag_engine = rag_engine
        self.recovery_attempts = 0
        self.max_attempts = 2
    
    def attempt_broader_search(self, original_query: str, failed_category: str) -> Optional[RetrievalResult]:
        """
        특정 카테고리 검색이 실패했을 때 전체 카테고리에서 재검색을 시도한다
        
        Args:
            original_query: 원본 검색 쿼리
            failed_category: 실패한 카테고리
        
        Returns:
            새로운 검색 결과 또는 None
        """
        print(f"Recovery: {failed_category} 카테고리에서 실패하여 전체 검색을 시도한다.")
        
        # 전체 카테고리에서 검색
        result = self.rag_engine.search(original_query, category=None, top_k=3)
        
        return result if result.chunks else None
    
    def reformulate_query(self, original_query: str) -> str:
        """
        검색 쿼리를 재구성하여 더 나은 결과를 얻는다
        
        Args:
            original_query: 원본 검색 쿼리
        
        Returns:
            재구성된 쿼리
        """
        print("Recovery: 쿼리를 재구성한다.")
        
        # LLM을 사용하여 쿼리를 더 구체적으로 재구성한다
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "사용자의 질문을 커피 메뉴, 레시피, 가격에 관한 더 구체적인 검색 쿼리로 변환하라. 핵심 키워드를 포함하여 20자 이내로 작성하라."
                },
                {
                    "role": "user",
                    "content": original_query
                }
            ],
            temperature=0.3
        )
        
        reformulated = response.choices[0].message.content.strip()
        print(f"재구성된 쿼리: {reformulated}")
        
        return reformulated
    
    def generate_fallback_answer(self, query: str) -> str:
        """
        모든 복구 시도가 실패했을 때 대체 답변을 생성한다
        
        Args:
            query: 원본 질문
        
        Returns:
            대체 답변
        """
        return f"죄송하지만 '{query}'에 대한 정확한 정보를 찾지 못했다. 메뉴판을 확인하시거나 직원에게 문의해 주시기 바란다."
    
    def reset_attempts(self):
        """복구 시도 카운터를 초기화한다"""
        self.recovery_attempts = 0

# 복구 시스템 인스턴스 생성
recovery_system = RecoverySystem(rag_engine)
print("복구 시스템이 초기화되었다.")

복구 시스템이 초기화되었다.


## 9. Feedback 시스템 구현

In [15]:
class FeedbackSystem:
    """피드백 시스템 클래스"""
    
    def __init__(self):
        self.feedback_history = []
    
    def evaluate_answer(self, query: str, answer: str, retrieval_result: RetrievalResult) -> float:
        """
        LLM을 사용하여 답변의 품질을 평가한다
        
        Args:
            query: 사용자 질문
            answer: 생성된 답변
            retrieval_result: 검색 결과
        
        Returns:
            0.0 ~ 1.0 사이의 평가 점수
        """
        # 참조 문서 내용 추출
        reference_texts = "\n".join([chunk.content for chunk in retrieval_result.chunks[:2]])
        
        # LLM을 사용한 답변 평가
        evaluation_prompt = f"""
다음 커피 키오스크 질문과 답변을 평가하라.

질문: {query}

답변: {answer}

참조 자료:
{reference_texts}

평가 기준:
1. 답변이 질문에 직접적으로 대응하는가?
2. 답변이 참조 자료의 내용과 일치하는가?
3. 답변이 명확하고 이해하기 쉬운가?
4. 답변이 커피 키오스크 맥락에 적합한가?

0.0부터 1.0 사이의 점수만 출력하라. (예: 0.85)
"""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 답변 품질을 평가하는 전문가다. 숫자 점수만 반환하라."},
                {"role": "user", "content": evaluation_prompt}
            ],
            temperature=0.1
        )
        
        score_text = response.choices[0].message.content.strip()
        score = float(score_text)
        score = max(0.0, min(1.0, score))
        
        # 피드백 기록 저장
        self.feedback_history.append({
            "query": query,
            "answer": answer,
            "score": score,
            "timestamp": datetime.now()
        })
        
        return score
    
    def get_improvement_suggestions(self, low_score_threshold: float = 0.6) -> List[str]:
        """
        낮은 점수를 받은 답변들을 분석하여 개선 제안을 생성한다
        
        Args:
            low_score_threshold: 낮은 점수로 간주할 임계값
        
        Returns:
            개선 제안 리스트
        """
        low_score_items = [item for item in self.feedback_history if item["score"] < low_score_threshold]
        
        if not low_score_items:
            return ["모든 답변이 양호한 품질을 유지하고 있다."]
        
        suggestions = [
            f"{len(low_score_items)}개의 답변이 개선이 필요하다.",
            "지식 베이스에 더 많은 정보를 추가하는 것을 고려하라.",
            "검색 쿼리 재구성 로직을 개선하라."
        ]
        
        return suggestions
    
    def get_average_score(self) -> float:
        """전체 답변의 평균 점수를 계산한다"""
        if not self.feedback_history:
            return 0.0
        return sum(item["score"] for item in self.feedback_history) / len(self.feedback_history)

# 피드백 시스템 인스턴스 생성
feedback_system = FeedbackSystem()
print("피드백 시스템이 초기화되었다.")

피드백 시스템이 초기화되었다.


## 10. Router RAG Agent 통합 구현

In [16]:
class RouterRAGAgent:
    """Router RAG Agent 메인 클래스"""
    
    def __init__(
        self,
        rag_engine: RAGEngine,
        memory: ConversationMemory,
        validator: ValidationSystem,
        recovery: RecoverySystem,
        feedback: FeedbackSystem
    ):
        self.rag_engine = rag_engine
        self.memory = memory
        self.validator = validator
        self.recovery = recovery
        self.feedback = feedback
        
        # 시스템 프롬프트를 메모리에 추가
        self.memory.add_message(
            "system",
            "당신은 커피 키오스크의 친절한 주문 도우미다. 고객의 질문에 정확하고 친절하게 답변하라."
        )
    
    def route_and_retrieve(self, user_query: str) -> tuple[RetrievalResult, str]:
        """
        사용자 질문을 라우팅하고 관련 정보를 검색한다
        
        Returns:
            (검색 결과, 사용된 라우트)
        """
        # Function Calling을 사용하여 적절한 도구 선택
        messages = self.memory.get_context()
        messages.append({"role": "user", "content": user_query})
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=TOOLS,
            tool_choice="auto"
        )
        
        # 도구 호출 여부 확인
        response_message = response.choices[0].message
        
        if response_message.tool_calls:
            # 첫 번째 도구 호출 처리
            tool_call = response_message.tool_calls[0]
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"선택된 라우트: {function_name}")
            print(f"검색 쿼리: {function_args['query']}")
            
            # 해당 도구 실행
            retrieval_result = TOOL_FUNCTIONS[function_name](function_args["query"])
            
            return retrieval_result, function_name
        
        # 도구를 선택하지 않은 경우 기본 전체 검색
        print("기본 라우트: 전체 검색")
        retrieval_result = self.rag_engine.search(user_query, category=None, top_k=3)
        return retrieval_result, "general_search"
    
    def generate_answer(self, user_query: str, retrieval_result: RetrievalResult) -> str:
        """
        검색 결과를 바탕으로 최종 답변을 생성한다
        
        Args:
            user_query: 사용자 질문
            retrieval_result: RAG 검색 결과
        
        Returns:
            생성된 답변
        """
        # 검색된 문서를 컨텍스트로 구성
        context_parts = []
        for i, chunk in enumerate(retrieval_result.chunks[:3]):
            context_parts.append(f"[참조 {i+1}] {chunk.content}")
        context = "\n\n".join(context_parts)
        
        # 답변 생성 프롬프트
        generation_prompt = f"""
다음 참조 정보를 바탕으로 고객의 질문에 답변하라.

고객 질문: {user_query}

참조 정보:
{context}

답변 작성 지침:
1. 참조 정보의 내용만을 사용하여 답변하라
2. 친절하고 명확한 어조로 작성하라
3. 불필요한 정보는 제외하라
4. 2-3문장으로 간결하게 답변하라
"""
        
        messages = self.memory.get_context(include_system=True)
        messages.append({"role": "user", "content": generation_prompt})
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.3
        )
        
        answer = response.choices[0].message.content.strip()
        return answer
    
    def process_query(self, user_query: str) -> AgentResponse:
        """
        사용자 질문을 처리하고 최종 응답을 생성한다
        
        Args:
            user_query: 사용자 질문
        
        Returns:
            AgentResponse 객체
        """
        print(f"\n{'='*60}")
        print(f"사용자 질문: {user_query}")
        print(f"{'='*60}")
        
        # 1. 사용자 메시지를 메모리에 추가
        self.memory.add_message("user", user_query)
        
        # 2. 라우팅 및 검색
        retrieval_result, route_used = self.route_and_retrieve(user_query)
        
        # 3. 검색 결과 검증
        is_valid, validation_message = self.validator.validate_retrieval(retrieval_result)
        print(f"검색 검증: {validation_message}")
        
        # 4. 검증 실패 시 Recovery 메커니즘 실행
        if not is_valid:
            print("Recovery 프로세스 시작...")
            
            # 전체 검색 시도
            retrieval_result = self.recovery.attempt_broader_search(user_query, route_used)
            
            # 여전히 실패하면 쿼리 재구성
            if retrieval_result is None or not retrieval_result.chunks:
                reformulated_query = self.recovery.reformulate_query(user_query)
                retrieval_result, route_used = self.route_and_retrieve(reformulated_query)
            
            # 모든 시도 실패 시 대체 답변
            if retrieval_result is None or not retrieval_result.chunks:
                answer = self.recovery.generate_fallback_answer(user_query)
                self.memory.add_message("assistant", answer)
                
                return AgentResponse(
                    answer=answer,
                    route_used=route_used,
                    sources=[],
                    confidence=0.0,
                    validation_passed=False,
                    feedback_score=None
                )
        
        # 5. 답변 생성
        answer = self.generate_answer(user_query, retrieval_result)
        print(f"\n생성된 답변: {answer}")
        
        # 6. 답변 검증
        answer_valid, answer_validation_msg = self.validator.validate_answer(answer, retrieval_result)
        print(f"답변 검증: {answer_validation_msg}")
        
        # 7. 신뢰도 계산
        confidence = self.validator.calculate_confidence(retrieval_result)
        print(f"신뢰도: {confidence:.2f}")
        
        # 8. 피드백 평가
        feedback_score = self.feedback.evaluate_answer(user_query, answer, retrieval_result)
        print(f"피드백 점수: {feedback_score:.2f}")
        
        # 9. 답변을 메모리에 추가
        self.memory.add_message("assistant", answer, metadata={
            "route": route_used,
            "confidence": confidence,
            "feedback_score": feedback_score
        })
        
        # 10. 최종 응답 생성
        sources = [chunk.id for chunk in retrieval_result.chunks[:3]]
        
        return AgentResponse(
            answer=answer,
            route_used=route_used,
            sources=sources,
            confidence=confidence,
            validation_passed=answer_valid,
            feedback_score=feedback_score
        )

# Router RAG Agent 인스턴스 생성
agent = RouterRAGAgent(
    rag_engine=rag_engine,
    memory=memory,
    validator=validator,
    recovery=recovery_system,
    feedback=feedback_system
)

print("\nRouter RAG Agent가 초기화되었다.")


Router RAG Agent가 초기화되었다.


## 11. 에이전트 테스트

다양한 시나리오로 Router RAG Agent를 테스트한다.

In [17]:
# 테스트 질문 리스트
test_queries = [
    "아메리카노는 어떤 커피인가요?",
    "카페라떼 만드는 방법을 알려주세요",
    "카푸치노 가격이 얼마인가요?",
    "녹차라떼에는 어떤 재료가 들어가나요?"
]

# 각 질문에 대해 에이전트 실행
for query in test_queries:
    response = agent.process_query(query)
    print(f"\n최종 응답:")
    print(f"  - 답변: {response.answer}")
    print(f"  - 사용된 라우트: {response.route_used}")
    print(f"  - 참조 소스: {', '.join(response.sources)}")
    print(f"  - 신뢰도: {response.confidence:.2f}")
    print(f"  - 검증 통과: {response.validation_passed}")
    print(f"  - 피드백 점수: {response.feedback_score:.2f}")
    print()


사용자 질문: 아메리카노는 어떤 커피인가요?
선택된 라우트: search_menu_info
검색 쿼리: 아메리카노
검색 검증: 검색 결과가 유효하다

생성된 답변: 아메리카노는 에스프레소에 물을 추가한 커피로, 진하고 깊은 맛이 특징입니다. HOT과 ICE 옵션이 있어 취향에 맞게 선택하실 수 있습니다.
답변 검증: 답변이 유효하다
신뢰도: 0.35
피드백 점수: 1.00

최종 응답:
  - 답변: 아메리카노는 에스프레소에 물을 추가한 커피로, 진하고 깊은 맛이 특징입니다. HOT과 ICE 옵션이 있어 취향에 맞게 선택하실 수 있습니다.
  - 사용된 라우트: search_menu_info
  - 참조 소스: menu_001, menu_003
  - 신뢰도: 0.35
  - 검증 통과: True
  - 피드백 점수: 1.00


사용자 질문: 카페라떼 만드는 방법을 알려주세요
선택된 라우트: search_recipe_info
검색 쿼리: 카페라떼 만드는 방법
검색 검증: 검색 결과가 유효하다

생성된 답변: 카페라떼를 만들려면 먼저 에스프레소 샷 2개를 추출합니다. 그 다음, 우유 200ml를 스티밍하여 65-70도 정도로 데운 후, 부드럽게 에스프레소에 붓습니다. 이렇게 하면 맛있는 카페라떼가 완성됩니다!
답변 검증: 답변이 유효하다
신뢰도: 0.39
피드백 점수: 1.00

최종 응답:
  - 답변: 카페라떼를 만들려면 먼저 에스프레소 샷 2개를 추출합니다. 그 다음, 우유 200ml를 스티밍하여 65-70도 정도로 데운 후, 부드럽게 에스프레소에 붓습니다. 이렇게 하면 맛있는 카페라떼가 완성됩니다!
  - 사용된 라우트: search_recipe_info
  - 참조 소스: recipe_002, recipe_003
  - 신뢰도: 0.39
  - 검증 통과: True
  - 피드백 점수: 1.00


사용자 질문: 카푸치노 가격이 얼마인가요?
선택된 라우트: search_price_info
검색 쿼리: 카푸치노 가격
검색 검증: 검색 결과가 유효하다


## 12. 대화형 인터페이스

In [20]:
def chat_interface():
    """대화형 인터페이스 함수"""
    print("\n" + "="*60)
    print("커피 키오스크 Router RAG Agent")
    print("종료하려면 'quit' 또는 'exit'를 입력하세요.")
    print("="*60 + "\n")
    
    while True:
        user_input = input("\n고객: ").strip()
        
        if user_input.lower() in ['quit', 'exit', '종료']:
            print("\n대화를 종료한다.")
            break
        
        if not user_input:
            continue
        
        response = agent.process_query(user_input)
        print(f"\n에이전트: {response.answer}")
        print(f"(신뢰도: {response.confidence:.2f}, 피드백: {response.feedback_score:.2f})")

# 대화 시작 (주석을 제거하여 실행)
chat_interface()


커피 키오스크 Router RAG Agent
종료하려면 'quit' 또는 'exit'를 입력하세요.




고객:  아메리카노 2잔 주문



사용자 질문: 아메리카노 2잔 주문
기본 라우트: 전체 검색
검색 검증: 검색 결과가 유효하다

생성된 답변: 아메리카노 2잔 주문해주셨군요! HOT 아메리카노는 4,500원, ICE 아메리카노는 5,000원입니다. 오후 2시~5시 사이에 주문하시면 20% 할인 혜택도 적용됩니다. 어떤 종류로 드릴까요?
답변 검증: 답변이 유효하다
신뢰도: 0.53
피드백 점수: 0.95

에이전트: 아메리카노 2잔 주문해주셨군요! HOT 아메리카노는 4,500원, ICE 아메리카노는 5,000원입니다. 오후 2시~5시 사이에 주문하시면 20% 할인 혜택도 적용됩니다. 어떤 종류로 드릴까요?
(신뢰도: 0.53, 피드백: 0.95)



고객:  아이스 아메리카노



사용자 질문: 아이스 아메리카노
선택된 라우트: search_price_info
검색 쿼리: 아이스 아메리카노 가격
검색 검증: 검색 결과가 유효하다

생성된 답변: 아이스 아메리카노는 5,000원입니다. 이번 주 프로모션으로 오후 2시~5시 사이에 구매하시면 20% 할인 혜택을 받으실 수 있습니다. 추가 사이즈 업그레이드 시 500원이 추가됩니다.
답변 검증: 답변이 유효하다
신뢰도: 0.53
피드백 점수: 1.00

에이전트: 아이스 아메리카노는 5,000원입니다. 이번 주 프로모션으로 오후 2시~5시 사이에 구매하시면 20% 할인 혜택을 받으실 수 있습니다. 추가 사이즈 업그레이드 시 500원이 추가됩니다.
(신뢰도: 0.53, 피드백: 1.00)



고객:  레귤러 사이즈로



사용자 질문: 레귤러 사이즈로
기본 라우트: 전체 검색
검색 검증: 검색 결과의 관련성이 낮다 (최고 점수: 0.27)
Recovery 프로세스 시작...
Recovery: general_search 카테고리에서 실패하여 전체 검색을 시도한다.

생성된 답변: 레귤러 사이즈 아이스 아메리카노는 5,000원입니다. 주문해주신 내용으로 준비해드리겠습니다. 감사합니다!
답변 검증: 답변이 유효하다
신뢰도: 0.26
피드백 점수: 1.00

에이전트: 레귤러 사이즈 아이스 아메리카노는 5,000원입니다. 주문해주신 내용으로 준비해드리겠습니다. 감사합니다!
(신뢰도: 0.26, 피드백: 1.00)



고객:  quit



대화를 종료한다.


## 13. 성능 분석 및 리포트

In [21]:
# 피드백 시스템 통계
print("\n" + "="*60)
print("성능 분석 리포트")
print("="*60)

avg_score = feedback_system.get_average_score()
print(f"\n평균 피드백 점수: {avg_score:.2f}")

suggestions = feedback_system.get_improvement_suggestions(low_score_threshold=0.7)
print("\n개선 제안:")
for i, suggestion in enumerate(suggestions, 1):
    print(f"  {i}. {suggestion}")

print(f"\n대화 메모리 상태: {memory.get_summary()}")

print("\n" + "="*60)
print("튜토리얼 1: Router RAG Agent 완료")
print("="*60)


성능 분석 리포트

평균 피드백 점수: 0.99

개선 제안:
  1. 모든 답변이 양호한 품질을 유지하고 있다.

대화 메모리 상태: 총 10개의 메시지 (사용자: 5, 에이전트: 5)

튜토리얼 1: Router RAG Agent 완료
